In [1]:
import requests      # Библиотека для отправки запросов
import numpy as np   # Библиотека для матриц, векторов и линала
import pandas as pd  # Библиотека для табличек
import time          # Библиотека для тайм-менеджмента
from bs4 import BeautifulSoup
import re
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from fake_useragent import UserAgent
from tqdm import tqdm
from webdriver_manager.firefox import GeckoDriverManager
from datetime import datetime
from selenium.webdriver.common.by import By
from sys import platform
import nltk
from nltk.tokenize import TweetTokenizer
from nltk.tokenize.treebank import TreebankWordDetokenizer
from nltk.corpus import stopwords
from string import punctuation
nltk.download("stopwords")

[nltk_data] Downloading package stopwords to /home/victor/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [2]:
print(platform)

linux


!pip install bs4
!pip install selenium
!pip install fake-useragent
!pip install webdriver-manager
!pip install nltk

In [3]:
MonthDict={ "января" : 1,
      "февраля": 2,
       "марта": 3,
       "апреля": 4,
       "мая": 5,
       "июня": 6,
       "июля": 7,
       "августа": 8,
       "сентября": 9,
       "октября": 10,
       "ноября": 11,
       "декабря": 12
}

In [4]:
#обозначили сайт, проверили, чтобы 200 ок
page_link = 'https://www.facebook.com/groups/realtor.russia'
response = requests.get(page_link, headers={'User-Agent': UserAgent().chrome})
response

<Response [200]>

In [5]:
driver = webdriver.Firefox(executable_path=GeckoDriverManager().install())

[WDM] - Getting latest mozilla release info for v0.28.0
[WDM] - Trying to download new driver from https://github.com/mozilla/geckodriver/releases/download/v0.28.0/geckodriver-v0.28.0-linux64.tar.gz
[WDM] - Driver has been saved in cache [/home/victor/.wdm/drivers/geckodriver/linux64/v0.28.0]


#две версии вызова робобраузера, для винды и для линукса. 
#PS. В винде указан путь, который верен только для моей локалки
if 'win' in platform:
    from selenium.webdriver.common.desired_capabilities import DesiredCapabilities

    cap = DesiredCapabilities().FIREFOX
    cap["marionette"] = False
    options = Options()
    options.binary_location = "C:\Program Files (x86)\Google\Chrome\Application\chrome.exe"
    driver = webdriver.Chrome(chrome_options=options, executable_path=r"C:\Users\matyukhinaa\Downloads\chromedriver_win32\chromedriver.exe", )
elif platform == 'linux':
    driver = webdriver.Firefox(executable_path=GeckoDriverManager().install())
    

In [6]:
#вызов нужного сайта
driver.get(page_link)
time.sleep(6)
ok_button = driver.find_elements_by_xpath("//div[contains(text(), 'OK')]")
try:
    ok_button[0].click()
except:
    pass

In [7]:
#Количество скроллов вниз, чем больше, тем больше получим записей. Но дольше будет работать
times_scroll = 50

In [8]:
#Цикл для повторения прокрутки
for i in tqdm(range(times_scroll)):
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    if i%5==0: #Здесь каждую пятую прокрутку цикл засыпает на 1 секунду, чтобы страница прогрузилась
               #Если убрать эту строчку, может не дорабатывать до конца и прогружать меньше
        time.sleep(1)

100%|██████████| 50/50 [00:10<00:00,  4.70it/s]


In [9]:
#Костыли для открытия комментов
buttons = driver.find_elements_by_xpath("//div[contains(text(), 'Ещё')]")
for i in tqdm(range(len(buttons))):
    try: 
        buttons[i].click()
    except:
        continue

#Костыли, созданные, чтобы открыть все длинные посты. К сожалению к двум циклам пришлось прибегнуть, так как
#часть кнопок не нажимаются с первого раза из-за того, что сайт недогрузился или не доскролился до нужной кнопки            
for i in range(2):       
    should_restart = True
    while should_restart:
        should_restart = False
        buttons = driver.find_elements_by_xpath("//span[contains(text(), 'Показать ещё')]")
        for i in range(len(buttons)):
            try: 
                buttons[i].click()
            except:
                should_restart = True
                break

buttons = driver.find_elements_by_xpath("//span[contains(text(), 'ответ')]")
for i in tqdm(range(len(buttons))):
    try: 
        buttons[i].click()
    except:
        continue
buttons = driver.find_elements_by_xpath("//span[contains(text(), 'ответов')]")
for i in tqdm(range(len(buttons))):
    try: 
        buttons[i].click()
    except:
        continue

100%|██████████| 23/23 [00:11<00:00,  1.98it/s]


In [10]:
# Чтение HTML Страницы
selen_page = driver.page_source
soup = BeautifulSoup(selen_page,'html.parser')

In [11]:
posts = (soup.findAll('div', attrs = {'class':"kvgmc6g5 cxmmr5t8 oygrvhab hcukyx3x c1et5uql ii04i59q"}))
b = soup.findAll('span', attrs = {'class':"gpro0wi8 j1lvzwm4 stjgntxs ni8dbmo4 q9uorilb"})
#текст постов и время размещения *не спрашивайте почему b*

In [12]:
#from datetime import timedelta
import datetime
import time

In [13]:
(datetime.datetime.now() - datetime.timedelta (days = 1)).day

9

In [14]:
#Загрузка сегодняшней даты в нужном формате
day_td = str(datetime.datetime.now().day)
day_ytd = str((datetime.datetime.now() - datetime.timedelta (days = 1)).day)
month_td = list(MonthDict.keys())[list(MonthDict.values()).index(datetime.datetime.now().month)]
year_td = str(datetime.datetime.now().year)
date_td = day_td+month_td+year_td
date_ytd = day_ytd+month_td+year_td
#Куча ИФ циклов для чистки и преобразования данных о времени
parsed_ls = []
#Первый цикл, для создания листа, где не будет лишних знаков
for i in tqdm(range(len(b))):
    parsed = (b[i].text.replace('=','').replace('·','').replace('\xa0', ''))
    if b[i].text != '':
        parsed_ls.append(parsed)
#Выкидывание ненужной информации и замена часов, минут и дней на дату
for i in tqdm(range(len(parsed_ls))):
    parsed_ls[i] = parsed_ls[i].replace(' ', '')
    parsed_ls[i] = parsed_ls[i].replace('г.Москва', '')
    parsed_ls[i] = parsed_ls[i].replace('г.', '')
    if 'ч.' in parsed_ls[i]:
        parsed_ls[i] = date_td
    if 'мин.' in parsed_ls[i]:
        parsed_ls[i] = date_td
    if 'Вчера' in parsed_ls[i]:
        parsed_ls[i] = date_ytd
#Перевод дат в формат д/м/г
for i in tqdm(range(len(parsed_ls))):
    if (parsed_ls[i][-3]==':')==True:
        parsed_ls[i]= parsed_ls[i][:-6]+year_td
    if parsed_ls[i][-4:-2]!='20':
        parsed_ls[i] = parsed_ls[i]+parsed_ls[i-1][-4:]
    if parsed_ls[i][:2].isnumeric():
        month = str(MonthDict[parsed_ls[i][2:-4]])
        date = (parsed_ls[i][:2])
        year = (parsed_ls[i][-4:])
        parsed_ls[i] = date+'/'+month+'/'+year
    if parsed_ls[i][:2].isnumeric()==False:
        month = str(MonthDict[parsed_ls[i][1:-4]])
        date = (parsed_ls[i][:1])
        year = (parsed_ls[i][-4:])
        parsed_ls[i] = date+'/'+month+'/'+year

100%|██████████| 55/55 [00:00<00:00, 116921.80it/s]


In [15]:
#перевод дат в формат datetime, перевод листа с хтмл кодом в текстовый лист
dt_ls = []
for date in (parsed_ls):
    dt_ls.append(datetime.datetime.strptime(date, '%d/%m/%Y'))
for i in range(len(posts)):
    posts[i] = posts[i].text

In [16]:
#Создание датафрейма
Dict = {'time': dt_ls[:len(posts)], 'post': posts}
df = pd.DataFrame.from_dict(Dict)

In [27]:
new_cl = 'd2edcug0 hpfvmrgz qv66sw1b c1et5uql gk29lw5a a8c37x1j keod5gw0 nxhoafnm aigsh9s9 d9wwppkn fe6kdd0r mau55g9w c8b282yb hrzyx87i jq4qci2q a3bd9o3v knj5qynh m9osqain'

In [28]:
#считывание комментариев и количества комментариев под каждым постом
comms = (soup.findAll('div', attrs = {'class':"tw6a2znq sj5x9vvc d1544ag0 cxgpxx05"}))
#comm_num_cl = 'd2edcug0 hpfvmrgz qv66sw1b c1et5uql gk29lw5a a8c37x1j keod5gw0 nxhoafnm aigsh9s9 d9wwppkn fe6kdd0r mau55g9w c8b282yb hrzyx87i jq4qci2q a3bd9o3v knj5qynh m9osqain'
comm_num = (soup.findAll('span', attrs = {'class':new_cl}))

In [30]:
#выкидываем кол-во репостов
comm_num_ls = []
for i in tqdm(range(len(comm_num))):
    if 'Комментарии' in comm_num[i].text:
        comm_num_ls.append(int(comm_num[i].text.replace('Комментарии: ', '')))
#почти всегда получается так, что посты без комментов технически сьедают комменты,
#а нижние посты остаются без них. Добьем лист нулями. Общая картина текстов не теряется
if len(comm_num_ls)<len(df):
    for i in range(len(df)-len(comm_num_ls)):
        comm_num_ls.append(0)

100%|██████████| 67/67 [00:00<00:00, 64013.30it/s]


In [31]:
#Добавляем колонку с кол-вом комментов
df['number_comments'] = np.array(comm_num_ls[:len(df)])

In [32]:
#токенизируем комменты, чтобы выкинуть мусор и убрать обращение по имени
#шумовые слова выкидываю сразу, чтобы не засорять датасет
tw = TweetTokenizer()
det = TreebankWordDetokenizer()
noise = stopwords.words('russian') + list(punctuation)
upnoise = [letter.upper() for letter in noise]
sum_noise = noise+upnoise+['.','»','«']
for i in tqdm(range(len(comms))):
    comms[i] = comms[i].text
    tokenized_example = (tw.tokenize(comms[i])[2:])
    #filtered_example = [word for word in tokenized_example if not word in noise]
    comms[i] = det.detokenize(tokenized_example)

100%|██████████| 1211/1211 [00:00<00:00, 6126.94it/s]


In [33]:
#разбиваем и клеим так, чтобы они были одним предложением. А также чтобы у каждого поста нужное кол-во
comms_ls = []
j = 0 
for i in tqdm(range(len(df))):
    comms_ls.append(comms[j:j+df.iloc[i,2]])
    j+=df.iloc[i,2]

#for i in range(len(comms_ls)):
 #   sentnc = ''
 #   for j in range(len(comms_ls[i])):
 #       sentnc += comms_ls[i][j]
  #      sentnc += ' '
 #   comms_ls[i] = sentnc
df['comment'] = comms_ls

100%|██████████| 46/46 [00:00<00:00, 40541.71it/s]


In [102]:
df.to_csv('fb.csv')

In [34]:
df

,time,post,number_comments,comment
0,2020-11-06,"Дети обанкротившихся бизнесменов, участвовавши...",26,[что не так? Оформляли на несовершеннолетних п...
1,2020-11-10,"Теперь все агенты недвижимости, обновившие при...",41,"[делает все, что бы подорвать основу агентства..."
2,2020-11-09,#вчемделоциан #цианнаглость #хватитциан#цианос...,28,"[, знакомая история, но мы успели вернуть матк..."
3,2020-11-08,"Доброе утро, коллеги. Ситуация из которой не з...",43,"[Вас агентство? При подписании двоу, ещё для к..."
4,2020-11-10,Коллеги.,4,"[Мишукова покупатель., Карпушкина, полностью с..."
5,2020-11-10,КАК ДОСТАВАТЬ ДОКУМЕНТЫ ИЗ РОСРЕЕСТРА?Зарегист...,30,"[Kutumov какого этого?, комиссию вперёд и рабо..."
6,2020-11-10,"Мой клиент покупал дом, в МО, порядка 12млн+.З...",28,"[дайте что просят и ладно, все же справки сраз..."
7,2020-11-10,"Добрый день!Готовлюсь к сделке,я со стороны пр...",34,"[на регистрацию отправили? До зачисления, Семе..."
8,2020-11-10,"Здравствуйте, уважаемые риэлторы!Я – режиссер,...",13,[Бухтоярова это же не значит что она не сущест...
9,2020-11-08,"Коллеги,добрый день!Поделитесь опытом пожалуйс...",17,"[игнор!, КвартирьерКоля любит Мамбу, нам идёт ..."


In [104]:
df.to_excel('exfb.xlsx')